In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import ta
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
import auto_trader

In [4]:
def adx(high, low, close, window=14):
    """
    Calculate the Average Directional Index (ADX).

    Parameters:
        high (pd.Series): Series of high prices
        low (pd.Series): Series of low prices
        close (pd.Series): Series of close prices
        window (int): The period over which to calculate the ADX (default is 14)

    Returns:
        pd.Series: ADX values indexed the same as the input data
    """
    # Calculate the components of the True Range
    tr = pd.Series(np.max([high - low, abs(high - close.shift(1)), abs(low - close.shift(1))], axis=0), index=close.index)
    tr_smoothed = tr.rolling(window=window, min_periods=1).sum()

    # Calculate +DM and -DM
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    minus_dm = minus_dm.abs()

    # Smooth +DM and -DM
    plus_dm_smoothed = plus_dm.rolling(window=window, min_periods=1).sum()
    minus_dm_smoothed = minus_dm.rolling(window=window, min_periods=1).sum()

    # Calculate +DI and -DI
    plus_di = 100 * plus_dm_smoothed / tr_smoothed
    minus_di = 100 * minus_dm_smoothed / tr_smoothed

    # Calculate DX
    dx = 100 * abs((plus_di - minus_di) / (plus_di + minus_di))

    # Calculate ADX
    adx = dx.rolling(window=window, min_periods=1).mean()

    return adx.rename('ADX')

In [5]:

ATR_LENGTH = 14
GRID_SIZE = 10
GRID_STEP = 1
trades = []
trades.append(('initialize', 0, time.time()))
def getData():
    data = auto_trader.basicPrepare()
    #Add ATR and ADX feature indicators to dataset
    data['ATR'] = AverageTrueRange(data['High Price'], data['Low Price'], data['Close Price'], window=ATR_LENGTH).average_true_range()
    data['ADX'] = adx(data['High Price'], data['Low Price'], data['Close Price'], window=14)
    return data



In [7]:
def checkPrice(grid, ADXLimit = 20, ATRMultiplier = 1, verbose = 0):
    """
    Check the price at a given index and make a trade if conditions are met
    ADXLimit: ADX value above which the bot will liquidate positions and not trade
    ATRMultiplier: Multiplier for the ATR value to determine grid size
    """
    data = getData()

    #get current price and ATR value
    atr = data['ATR'][0]
    current_price = data['Close Price'][0]
    if verbose:
        print('Current Price: ' + str(current_price))
        print('ADX: ' + str(data['ADX'][0]))

    #if trending, clear grid and wait
    if(data['ADX'][0] > ADXLimit):
        if verbose:
            print('Trending, clear grid and wait')
        auto_trader.sell_all() #liquidate position 
        grid['buy'].clear()
        grid['sell'].clear()
        return grid

    #if conditions are met, make a new grid
    if(len(grid['buy']) == 0 or len(grid['sell']) ==0 ):
        grid = create_grid(current_price, grid, atr, ATRMultiplier)
        if verbose:
            print('New Grid Created with ATR:' + str(atr) + '\n' +str(grid))

    #if price hits a level, buy or sell
    if current_price <= max(grid['buy']):
        auto_trader.buy()
        grid['buy'].remove(max(grid['buy']))
        if verbose:
            print('bought at ' + str(current_price))
        
        trades.append(('buy', current_price, time.time()))
    elif current_price >= min(grid['sell']):
        auto_trader.sell()
        grid['sell'].remove(min(grid['sell']))
        if verbose:
            print('sold at ' + str(current_price))
        trades.append(('sell', current_price, time.time()))


    return grid
def create_grid(current_price, grid, atr, ATRMultiplier = 1):
    step = atr*ATRMultiplier
    lower_bound = current_price - GRID_SIZE*step/2
    upper_bound = current_price + GRID_SIZE*step/2
    
    for i in range(5):
        
        grid['buy'].append(lower_bound + i*step)
        grid['sell'].append(upper_bound - i*step)
        
    return grid

In [10]:
grid = {'buy': [], 'sell': []}
auto_trader.queue = auto_trader.build_queue()

if os.path.exists('gridTrades.csv'):
    df = pd.read_csv('gridTrades.csv')
    for index, row in df[['trade', 'price', 'time']].iterrows():
        trades.append([row['trade'], row['price'], row['time']])
print(trades)
while True:

    grid = checkPrice(grid, ADXLimit= 50, ATRMultiplier= 1, verbose = 1)
    df = pd.DataFrame(trades, columns = ['trade', 'price', 'time'])
    df['log'] = df['trade'] + ' at ' + df['price'].astype(str) + ' at ' + df['time'].astype(str)
    df.to_csv('gridTrades.csv')
    time.sleep(60)

[('initialize', 0, 1713575618.0094624), ['initialize', 0.0, 1713569476.2225442], ['initialize', 0.0, 1713569388.9837892], ['sell', 64481.67, 1713564671.7107592], ['sell', 64481.67, 1713564732.1850514], ['sell', 64481.67, 1713564792.6721146], ['buy', 64328.42, 1713565278.289673], ['sell', 63991.7, 1713570391.1720624], ['sell', 63991.7, 1713570452.1393242], ['sell', 63991.7, 1713570512.791005], ['sell', 63991.7, 1713570573.3766756], ['sell', 63991.7, 1713570633.9693124], ['buy', 63801.9, 1713570999.7008443], ['buy', 63801.9, 1713571061.5820713], ['buy', 63751.21, 1713571305.62342], ['buy', 63718.24, 1713572217.2341778], ['buy', 63521.95, 1713574344.3543794], ['buy', 63521.95, 1713574407.0229156], ['buy', 63521.95, 1713574469.212945]]
Current Price: 64210.74
ADX: 31.861355432033786
New Grid Created with ATR:35.92876698949278
{'buy': [64031.096165052535, 64067.024932042026, 64102.95369903152, 64138.882466021016, 64174.81123301051], 'sell': [64390.38383494746, 64354.45506795797, 64318.52630